# Feedforward Neural Networks

A full, general FNN function

In [1]:
import numpy as np
import copy
from typing import Callable

## Activation Functions

In [2]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def deriv_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

In [3]:
def tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x)+np.exp(-x))

def deriv_tanh(x):
    return 1 - np.power(tanh(x),2)

In [4]:
def relu(x):
    return x * (x>=0)

def deriv_relu(x):
    return (x>=0).astype("int")

In [5]:
def linear(x):
    return x

def deriv_linear(x):
    return np.ones(x.shape)

## Cost Functions

In [6]:
def mse(yhat, y):
    return np.power(yhat-y,2)

def deriv_mse(yhat, y):
    return 2*(yhat-y)

In [7]:
def cross_entropy(yhat, y):
    return -y * np.log(yhat) - (1-y)*np.log(1-yhat)

def deriv_cross_entropy(yhat, y):
    return -(y/yhat) + (1-y)/(1-yhat)

## Managing Activation and Cost Function Choice

In [8]:
# Function to manage which activation function to use
def get_activation(name: str) -> Callable:
    if name == "sigmoid":
        return sigmoid, deriv_sigmoid
    if name == "tanh":
        return tanh, deriv_tanh
    if name == "relu":
        return relu, deriv_relu
    if name == "linear":
        return linear, deriv_linear

# Function to manage which cost function to use
def get_cost(name: str) -> Callable:
    if name == "mse":
        return mse, deriv_mse
    if name == "ce":
        return cross_entropy, deriv_cross_entropy

## Weight Initialisation

In [9]:
def xavier_init(shape):
    num_input = shape[1]
    num_output = shape[0]
    W = np.random.randn(num_output, num_input) / num_input
    return W

def he_init(shape):
    num_input = shape[1]
    num_output = shape[0]
    W = np.random.randn(num_output, num_input) * np.sqrt(2/num_input)
    return W

def linear_init(shape):
    num_input = shape[1]
    num_output = shape[0]
    W = np.random.randn(num_output, num_input)/num_input
    return W

def get_init(name):
    if name == "sigmoid" or name == "tanh":
        return xavier_init
    if name == "relu":
        return he_init
    if name == "linear":
        return linear_init
    

## Forward Propagation

Note Z\[i+1\] is because we're storing X in the Z_list as well. i+1 should be understood as layer i!

In [10]:
def forward_pass(Z_list, W_list, B_list, activation_hidden,
                 activation_output, num_pass):
    """_summary_

    Args:
        Z_list (list[np.ndarray]): A list of num_features by num_obs
                                                 matrices, where the number of
                                                 features is the number of nodes
                                                 on the previous layer
        weight_list (list[np.ndarray]): A list of num_features_l by num_features_(l-1)
                                        matrices, where num_features_l is the number
                                        of features for the lth layer
        bias_list (list[np.ndarray]): _description_
        activation_hidden (_type_): _description_
        activation_output (_type_): _description_
        num_pass (_type_): _description_

    Returns:
        _type_: _description_
    """
    A_list = [Z_list[0]] #for use in backprop
    for i in range(num_pass):
        Z_list[i+1] = np.dot(W_list[i],A_list[i]) + B_list[i]
        if i < (num_pass-1):
            A_list.append(activation_hidden(Z_list[i+1]))
        elif i == (num_pass-1):
            A_list.append(activation_output(Z_list[i+1]))

    return Z_list, A_list

## Back Propagation

In [11]:
def back_pass(cost_deriv,
              Y,
              activation_output_deriv,
              activation_hidden_deriv,
              W_list,
              A_list,
              Z_list,
              num_pass,
              num_obs):
    # Docstring ends here
    # dZ_list = [cost_deriv(A_list[num_pass], Y) * activation_output_deriv(Z_list[num_pass])]
    dZ = np.multiply(cost_deriv(A_list[num_pass], Y), activation_output_deriv(Z_list[num_pass]))
    dW_list = [np.dot(dZ, A_list[num_pass-1].T)/num_obs]
    dB_list = [np.sum(dZ, axis=1, keepdims=True)/num_obs]
    for i in range(num_pass-1, 0, -1):
        # dZ_list = [np.dot(W_list[i].T, dZ_list[0]) * activation_hidden_deriv(Z_list[i])] + dZ_list
        dZ = np.multiply(np.dot(W_list[i].T, dZ), activation_output_deriv(Z_list[i]))
        dW_list = [np.dot(dZ, A_list[i-1].T)/num_obs] + dW_list
        dB_list = [np.sum(dZ, axis=1, keepdims=True)/num_obs] + dB_list
    return dW_list, dB_list

## A Feedforward Neural Network Function

In [12]:
def fnn(X, Y, hidden, learn_rate=0.03, hidden_activation="relu",
        output_activation="linear", loss="mse", batch_size=2, tol=0.0001, max_iter=10000):
    """
    A feedforward neural network function that allows an arbitary number of 
    hidden layers and 

    Args:
        X (np.ndarray): A num_features by num_obs matrix
        Y (np.ndarray): A num_obs by num_outputs matrix
        hidden (list[int]): A list of length num_hidden_layers where each
                            entry describes the number of nodes for the nth
                            hidden layer.
        learn_rate (float): Learning rate of the network.
        hidden_activation (str): Activation function for hidden layers.
        output_activation (str): Activation function for output layer.
        loss: loss function
        tol (float): Minimum difference for ending training loop.
        max_iter (int): Maximum number of training iterations.
    """
    
    # Data will be aligned in a different format to the network
    X = X.T
    Y = Y.T
    
    # Initialise network: core numbers
    shape_X = X.shape
    num_obs = shape_X[1]
    indices = list(range(num_obs))
    num_features = shape_X[0]
    shape_Y = Y.shape
    num_output = shape_Y[0]
    num_hidden = len(hidden)
    num_pass = num_hidden + 1 #passes between layers
    
    # Initialise network: weighted outputs of nodes Z
    Z_list = [X]
    for i in range(num_hidden):
        Z_list.append(np.zeros((hidden[i], num_obs)))
    Z_list.append(np.zeros((num_output, num_obs)))
    
    # Initialise network: weights W:
    hidden_init, output_init = get_init(hidden_activation), get_init(output_activation)
    W_list =  [hidden_init((hidden[0], num_features))]
    for i in range(1, num_hidden):
        W_list.append(hidden_init((hidden[i], hidden[i-1])))
    W_list.append(output_init((num_output, hidden[num_hidden-1])))
    
    # Initialise network: biases B
    B_list = []
    for i in range(num_hidden):
        B_list.append(np.zeros((hidden[i],1)))
    B_list.append(np.zeros((num_output, 1)))
    
    # Initalise network: activation + cost funcs and their derivatives
    activation_hidden, activation_hidden_deriv = get_activation(hidden_activation)
    activation_output, activation_output_deriv = get_activation(output_activation)
    cost_func, cost_deriv = get_cost(loss)
    
    # Training loop
    iter = 0
    diff = np.inf
    while (iter <= max_iter and diff > tol):
        # Compute forward pass
        Z_list, A_list = forward_pass(Z_list=copy.deepcopy(Z_list),
                                      W_list=copy.deepcopy(W_list),
                                      B_list=copy.deepcopy(B_list),
                                      activation_hidden=activation_hidden,
                                      activation_output=activation_output,
                                      num_pass=num_pass)
        
        # Compute cost
        cost = cost_func(A_list[num_pass], Y) #individual costs
        
        # Backpropagate to find gradients
        batch_indices = list(np.random.choice(num_obs, batch_size, replace=False))
        dW_list, dB_list = back_pass(cost_deriv=cost_deriv,
                                     Y=copy.deepcopy(Y[:,batch_indices]),
                                     activation_output_deriv=activation_output_deriv,
                                     activation_hidden_deriv=activation_hidden_deriv,
                                     W_list=copy.deepcopy(W_list),
                                     A_list=copy.deepcopy([A[:,batch_indices] for A in A_list]),
                                     Z_list=copy.deepcopy([Z[:,batch_indices] for Z in Z_list]),
                                     num_pass=num_pass,
                                     num_obs=batch_size)
        
        
        # Check the gradients - slow but worthwhile
        epsilon = 1e-4
        cost_plus = []
        cost_minus = []
        num_grads = []
        diff_grads = []
        for i in range(num_pass):
            cost_plus.append(np.zeros(W_list[i].shape))
            cost_minus.append(np.zeros(W_list[i].shape))
            num_grads.append(np.zeros(W_list[i].shape))
            diff_grads.append(np.zeros(W_list[i].shape))
            for idx, x in np.ndenumerate(W_list[i]):
                W_plus = copy.deepcopy(W_list)
                W_plus[i][idx] = W_plus[i][idx] + epsilon
                W_minus = copy.deepcopy(W_list)
                W_minus[i][idx] = W_minus[i][idx] - epsilon
       
                _, A_plus = forward_pass(Z_list=copy.deepcopy(Z_list),
                                        W_list=copy.deepcopy(W_plus),
                                        B_list=copy.deepcopy(B_list),
                                        activation_hidden=activation_hidden,
                                        activation_output=activation_output,
                                        num_pass=num_pass)
                _, A_minus = forward_pass(Z_list=copy.deepcopy(Z_list),
                                        W_list=copy.deepcopy(W_minus),
                                        B_list=copy.deepcopy(B_list),
                                        activation_hidden=activation_hidden,
                                        activation_output=activation_output,
                                        num_pass=num_pass)
                cost_plus[i][idx] = np.sum(cost_func(A_plus[num_pass], Y))
                cost_minus[i][idx] = np.sum(cost_func(A_minus[num_pass], Y))
                num_grads[i][idx] = (cost_plus[i][idx] - cost_minus[i][idx])/(2*epsilon)
                diff_grads[i][idx] = np.abs(num_grads[i][idx] - dW_list[i][idx])
        
        # print(diff_grads)
        grad_tests = []
        for i in range(num_pass):
            grad_tests.append(np.isclose(num_grads[i], dW_list[i]))
        # print(grad_tests)
        
        # Update weights (w_new = w_old - learn_rate * gradient(w_old))
        W_old = copy.deepcopy(W_list)
        B_old = copy.deepcopy(B_list)
        new_diff = 0
        for i in range(num_pass):
            W_list[i] = W_list[i] - (learn_rate * num_grads[i]) #(learn_rate * dW_list[i])
            B_list[i] = B_list[i] - (learn_rate * dB_list[i])
            new_diff += np.sum(np.abs(W_list[i] - W_old[i])) + np.sum(np.abs(B_list[i] - B_old[i]))
        diff = new_diff
        # print(dW_list)
        # print(Z_list[num_pass])
        # print(A_list[num_pass])
        # print(A_list[num_pass].shape)
        # print(Y.shape)
        # print(cost_func)
        # print(cost)
        # print(W_list)
        # print(cost.shape)
        # print(np.sum(cost))

        # Increment iterations
        iter += 1
    
    # Predicted values of the network
    prediction = A_list[num_pass].T
    
    # Prediction function
    def fnn_pred_func(X):
        X = X.T,
        _, A_list = forward_pass(Z_list=[X] + [[]]*(num_hidden+1),
                                      W_list=copy.deepcopy(W_list),
                                      B_list=copy.deepcopy(B_list),
                                      activation_hidden=activation_hidden,
                                      activation_output=activation_output,
                                      num_pass=num_pass)
        return A_list[num_pass]
    
    # print(iter <= max_iter and diff > tol)
    # print(iter <= max_iter)
    # print(diff > tol)
    # print(num_obs)
    
    # Return Yhat
    return prediction, iter, diff, np.sum(cost), fnn_pred_func
    
    
    

## Example 1: Learning XOR

In [13]:
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])
Y = np.array([[0],[1],[1],[0]])

print(X)
print(Y)

[[0 0]
 [0 1]
 [1 0]
 [1 1]]
[[0]
 [1]
 [1]
 [0]]


In [32]:
# np.random.seed(42)
pred, iter, diff, cost, func = fnn(X=X,
                                   Y=Y,
                                   hidden=[2],
                                   learn_rate=0.9,
                                   batch_size=2,
                                   hidden_activation="sigmoid",
                                   output_activation="sigmoid",
                                   loss="mse",
                                   max_iter=1000)
print(np.round(pred,2))
print(pred.shape)
# print(pred)
# print(iter)
# print(diff)
# print(cost)

[[0.03]
 [0.95]
 [0.95]
 [0.05]]
(4, 1)


In [15]:
print(func(np.array([[0,1]])))
# print(func(np.array([[1,1]])))


[[[0.47427244]
  [0.47331199]]]


## Example 2: 

In [16]:
# import sklearn.datasets
# data_X, data_Y =  sklearn.datasets.load_breast_cancer(return_X_y=True, as_frame=False)

In [17]:
# import sklearn.model_selection
# indices = list(range((X.shape[0])))
# train_indices, test_indices = sklearn.model_selection.train_test_split(indices, test_size=0.2)

In [18]:
# _, _, _, _, func_bc = fnn(X=data_X[train_indices,],
#                        Y=data_Y[train_indices],
#                         hidden=[10,8,6],
#                         learn_rate=0.05,
#                         hidden_activation="sigmoid",
#                         output_activation="sigmoid",
#                         loss="mse",
#                         max_iter=10000)

In [19]:
# test_pred = func_bc(data_X[test_indices,])
# print(test_pred)
# print(Y[test_indices])
# # print(test_pred - Y[test_indices])
# test_pred.shape